In [203]:
import numpy as np

In [204]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [205]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다.

            gys = [output.grad for output in f.outputs] # ❶
            gxs = f.backward(*gys) # ❷
            if not isinstance(gxs, tuple): # ❸
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # ❹
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    # +=을 쓰지 않는 이유: 값을 덮어쓰는게(인플레이스) 아니라 복사하게끔 + 사용
                    # 인플레이스 연산:  연산에 대한 결괏값을 새로운 변수에 저장하는 것이 아닌
                    # 기존 데이터를 대체하는 것을 의미
                if x.creator is not None:
                    funcs.append(x.creator)

In [206]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)        # ❶ 별표를 붙여 언팩
        if not isinstance(ys, tuple): # ❷ 튜플이 아닌 경우 추가 지원
            ys = (ys,)

        outputs = [Variable(as_array(y)) for y in ys]  # Variable 형태로 되돌린다.
        for output in outputs:
            output.set_creator(self) # 출력 변수에 창조자를 설정한다.

        self.inputs  = inputs       # 입력 변수를 기억(보관)한다.
        self.outputs = outputs      # 출력도 저장한다.

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [207]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [208]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def add(x0, x1):
    f = Add()
    return f(x0, x1)

In [209]:
x = Variable(np.array(3.0))
y = add(x, x)
print(f'y {y.data}')
y.backward()
print(f'y.grad {y.grad}')
print(f'x.grad {x.grad}')

y 6.0
y.grad 1.0
x.grad 2.0


In [210]:
y = np.array(3.0)
print(f'y: {y} ({id(y)})')
x = y
print(f'x: {x} ({id(x)})')
x = x + np.array(1.0)
print(f'y: {y} ({id(y)})')
print(f'x: {x} ({id(x)})')

y: 3.0 (5650476144)
x: 3.0 (5650476144)
y: 3.0 (5650476144)
x: 4.0 (5617885936)


In [211]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
y = add(add(x, x), x)
y.backward()
print(x.grad) # 3.0이 나와야 된다.

2.0
5.0


In [212]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
x.cleargrad() # 미분값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad) # 3.0이 나와야 된다.

2.0
3.0
